<a href="https://colab.research.google.com/github/baduyne/llmprojects/blob/main/Hand_on_Fine_tuning_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install --upgrade pip
! pip install pert dataset

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
from datasets import load_dataset, Dataset

HF_USER = "baduyne"
PROJECT_NAME = "Book's Price"

# The run itself
RUN_NAME = "2025-07-07_12.53.07"
PROJECT_RUN_NAME = f"{PROJECT_NAME}-{RUN_NAME}"
REVISION = "71dc08b7f8c91e521ed510a5752f97187e415a2e" # or REVISION = None
FINETUNED_MODEL = f"{HF_USER}/{PROJECT_RUN_NAME}"


In [3]:
from datasets import load_dataset
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm_notebook
# ds = load_dataset("cogsci13/Amazon-Reviews-2023-Books-Meta")
# load data paralle processing
# Data
DATASET_NAME = "cogsci13/Amazon-Reviews-2023-Books-Meta"
raw_data = load_dataset(DATASET_NAME,split="full", cache_dir="./tmp_hf_cache")

Resolving data files:   0%|          | 0/28 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/28 [00:00<?, ?it/s]

In [4]:
len(raw_data)

4448181

In [5]:
# set up hugging face environtment
HF_USER = "baduyne"
PROJECT_NAME = "book'sPricePrediction"

| Task                       | Class                                |
| -------------------------- | ------------------------------------ |
| Causal LM                  | `AutoModelForCausalLM`               |
| Seq2Seq LM                 | `AutoModelForSeq2SeqLM`              |
| Classification             | `AutoModelForSequenceClassification` |
| Token Classification (NER) | `AutoModelForTokenClassification`    |
| Masked LM (BERT)           | `AutoModelForMaskedLM`               |
| Generic Encoder            | `AutoModel`                          |


In [2]:
# using transformer library being supported by hugging face 
from transformers import AutoTokenizer, BertTokenizer, AutoModelForCausalLM

Model_name = "meta-llama/Meta-Llama-3.1-8B"
Tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
vocab = Tokenizer.get_vocab()
vocab

{'[PAD]': 0,
 '[unused1]': 1,
 '[unused2]': 2,
 '[unused3]': 3,
 '[unused4]': 4,
 '[unused5]': 5,
 '[unused6]': 6,
 '[unused7]': 7,
 '[unused8]': 8,
 '[unused9]': 9,
 '[unused10]': 10,
 '[unused11]': 11,
 '[unused12]': 12,
 '[unused13]': 13,
 '[unused14]': 14,
 '[unused15]': 15,
 '[unused16]': 16,
 '[unused17]': 17,
 '[unused18]': 18,
 '[unused19]': 19,
 '[unused20]': 20,
 '[unused21]': 21,
 '[unused22]': 22,
 '[unused23]': 23,
 '[unused24]': 24,
 '[unused25]': 25,
 '[unused26]': 26,
 '[unused27]': 27,
 '[unused28]': 28,
 '[unused29]': 29,
 '[unused30]': 30,
 '[unused31]': 31,
 '[unused32]': 32,
 '[unused33]': 33,
 '[unused34]': 34,
 '[unused35]': 35,
 '[unused36]': 36,
 '[unused37]': 37,
 '[unused38]': 38,
 '[unused39]': 39,
 '[unused40]': 40,
 '[unused41]': 41,
 '[unused42]': 42,
 '[unused43]': 43,
 '[unused44]': 44,
 '[unused45]': 45,
 '[unused46]': 46,
 '[unused47]': 47,
 '[unused48]': 48,
 '[unused49]': 49,
 '[unused50]': 50,
 '[unused51]': 51,
 '[unused52]': 52,
 '[unused53]': 53

In [7]:
sequence = "Hi, my name is Duy"

token_sque = Tokenizer.tokenize(sequence)

In [8]:
token_sque

['Hi', ',', 'my', 'name', 'is', 'Du', '##y']

In [9]:
# convert words to ids which respect to dictionary
sequence_ids = Tokenizer.convert_tokens_to_ids(token_sque)
sequence_ids

[8790, 117, 1139, 1271, 1110, 12786, 1183]

In [10]:
sequence = Tokenizer.convert_ids_to_tokens(sequence_ids)
sequence

['Hi', ',', 'my', 'name', 'is', 'Du', '##y']

In [11]:
Tokenizer.pad

<bound method PreTrainedTokenizerBase.pad of BertTokenizer(name_or_path='bert-base-cased', vocab_size=28996, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)>

In [12]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")


2025-07-09 14:31:59.171873: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752071519.202702  232699 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752071519.211937  232699 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752071519.237232  232699 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752071519.237276  232699 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752071519.237279  232699 computation_placer.cc:177] computation placer alr

In [13]:
tokenizer.pad
tokenizer

BertTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [14]:
inputs = tokenizer("I love this!","hi", return_tensors="pt", padding = True, truncation= True, max_length= 50)
inputs

{'input_ids': tensor([[ 101, 1045, 2293, 2023,  999,  102, 7632,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [15]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

inputs = tokenizer(
    "I love this movie very very much", 
    "It's amazing!", 
    return_tensors="pt", 
    padding=True, 
    truncation=True, 
    max_length=10,
)
inputs

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


{'input_ids': tensor([[ 101, 1045, 2293, 2023, 3185,  102, 2009, 1005, 1055,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

```python
inputs = {
    "input_ids": tensor(...),
    "attention_mask": tensor(...)
}

# unpacking values of dic 
outputs = model(**inputs)
```

In [17]:
import numpy as np 
import torch.nn.functional as f
output  = model(**inputs)

class_name = ["negative", "positive"]
# np.argmax(output.logits, axis=0)

# softmax function
pro = f.softmax(output.logits, dim=1 )

# determine class 
class_name[torch.argmax(pro, dim = 1)]

'negative'

In [1]:
# load model LLAM 

model = AutoModelForCausalLM.from_pretrained(Model_name)

NameError: name 'AutoModelForCausalLM' is not defined